# Quantitative Pilot - 500 Randomly GSE

Quantitative analysis GSM per GSM (geometadb x geo NCBI XML)

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 20)

gsm_gse_geodb = pd.read_csv('/home/local/USHERBROOKE/frog2901/Documents/venv/JF_Frosi_Code_NGS-QC/gsm_gse_all_GEOdb.csv', sep=',' )




In [2]:
#GEOmetadb table from R - general (without filters) and not grouped

gsm_gse_geodb

,gsm,series_id,gpl
0,GSM1,GSE506,GPL4
1,GSM2,GSE506,GPL4
2,GSM3,GSE462,GPL5
3,GSM4,GSE462,GPL5
4,GSM5,GSE462,GPL5
5,GSM6,GSE462,GPL5
6,GSM7,GSE462,GPL5
7,GSM8,GSE462,GPL5
8,GSM9,GSE462,GPL5
9,GSM10,GSE462,GPL5


In [4]:
#from the column 'series_id', 500 GSE were selected randomly

#however, we have some rows which contain more than one GSE. So, to avoid problems with NGS-QC table match,
#just the first element in this case were selected.


gse_list = np.random.choice(gsm_gse_geodb['series_id'].unique(), 500).tolist() #get 500 GSE randomly
gse_list

gse_all = []
for i in gse_list:
    a = i.split(',') #split is necessary because there are some GSM that are related with more then one GSE
    gse_all.append(a[0]) #in the cases mentioned above, the code will get the first one

print(len(gse_all))


500


In [165]:
#Previously, in the merged table, GSE GSE121400 did not appear, because it is together with another GSE (2 SAMPLES).  
#So, was necessary split the column and get the first element to match with our list.


gsm_gse_geodb['GSE'] = gsm_gse_geodb['series_id'].str.split(',').str[0]


In [164]:
gsm_gse_geodb.head() #ok

,gsm,series_id,gpl,GSE
0,GSM1,GSE506,GPL4,GSE506
1,GSM2,GSE506,GPL4,GSE506
2,GSM3,GSE462,GPL5,GSE462
3,GSM4,GSE462,GPL5,GSE462
4,GSM5,GSE462,GPL5,GSE462


In [103]:
#grouping b splitedy column - the problem is: the first element, as GSE1000, sometimes, is the second element in some rows.
#because of this, we wont be able to count the all GSM related to this GSE - we need to pay atention when merged the tables

gsm_gse_geodb_grouped = gsm_gse_geodb.groupby(['GSE'])['gsm'].count().reset_index()
gsm_gse_geodb_grouped.head()


,GSE,gsm
0,GSE1,38
1,GSE10,4
2,GSE100,4
3,GSE10000,29
4,GSE100001,6


### Example Position Issue - GSE1000

In [ ]:
# gsm_gse_geodb[gsm_gse_geodb['GSE'].str.contains('GSE1000', na=False)]


# Find the 500 selected series in geometadb grouped 

In [114]:



gsm_gse_geodb_grouped_500 = gsm_gse_geodb_grouped[gsm_gse_geodb_grouped['GSE'].isin(gse_all)].reset_index(drop=True) 

gsm_gse_geodb_grouped_500.rename(columns={'gsm': 'samples_GEOdb'}, inplace=True) #499 rows...I do not know why

gsm_gse_geodb_grouped_500.head()



,GSE,samples_GEOdb
0,GSE100068,15
1,GSE100811,4
2,GSE101190,1
3,GSE101508,10
4,GSE101879,12


# XML from NCBI - 500 GSE selected above

In [9]:

import urllib.request
import lxml.html as lh
from bs4 import BeautifulSoup
import requests
import re
from time import sleep

#I've tried download the txt file to extract the information. However, in txt file there is no quantitative of samples by GSE, just by GPL
#I tried to parse "if Samples in td". However, the tr related with our information is too long, and the code was return a lot of text, not just the quantitative information
#because of this, I did a regex to be more specific.



regex = re.compile('Samples\s+\(\d+\)')  #get the sample (number)
gse_samples_ncbi =[]

for gse in gse_all:
    url = 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=' + gse
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'lxml')
    tr = soup.findAll("tr", {"valign" : "top"}) #ok    

    for td in tr:
        match = re.search(regex, td.text)
        if match:
            gse_samples_ncbi.append([gse, match.group(0)])
    
    sleep(2) #Ok

print(gse_samples_ncbi)

[['GSE76110', 'Samples (16)'], ['GSE114799', 'Samples (138)'], ['GSE38924', 'Samples (1)'], ['GSE91069', 'Samples (31)'], ['GSE6903', 'Samples (6)'], ['GSE125134', 'Samples (28)'], ['GSE121400', 'Samples (2)'], ['GSE66578', 'Samples (6)'], ['GSE13150', 'Samples (202)'], ['GSE123081', 'Samples (2)'], ['GSE19359', 'Samples (8)'], ['GSE41403', 'Samples (1)'], ['GSE98040', 'Samples (8)'], ['GSE5646', 'Samples (7)'], ['GSE72937', 'Samples (9)'], ['GSE40173', 'Samples (19)'], ['GSE16795', 'Samples (39)'], ['GSE83012', 'Samples (2)'], ['GSE117450', 'Samples (5)'], ['GSE51192', 'Samples (16)'], ['GSE50418', 'Samples (8)'], ['GSE63322', 'Samples (4)'], ['GSE107115', 'Samples (158)'], ['GSE7931', 'Samples (56)'], ['GSE8661', 'Samples (15)'], ['GSE49373', 'Samples (6)'], ['GSE26197', 'Samples (16)'], ['GSE4808', 'Samples (69)'], ['GSE12688', 'Samples (18)'], ['GSE8967', 'Samples (10)'], ['GSE89728', 'Samples (22)'], ['GSE88454', 'Samples (2)'], ['GSE105729', 'Samples (2)'], ['GSE73265', 'Samples 

In [10]:
#Check the length of our list - Ok

print(len(gse_samples_ncbi))

500


In [11]:
header = ['GSE', 'Samples_NCBI']

In [108]:
#Create a df from NCBI data

gse_samples_ncbi_df  = pd.DataFrame(gse_samples_ncbi, columns=header)

gse_samples_ncbi_df.head()

,GSE,Samples_NCBI
0,GSE76110,Samples (16)
1,GSE114799,Samples (138)
2,GSE38924,Samples (1)
3,GSE91069,Samples (31)
4,GSE6903,Samples (6)


In [166]:
#merging tables 

df_merged = pd.merge(gse_samples_ncbi_df, gsm_gse_geodb_grouped_500, on='GSE', how='left') 

df_merged

#samples_GEOdb is a int. we need to change the samples_NCBI type

,GSE,Samples_NCBI,samples_GEOdb
0,GSE76110,Samples (16),16
1,GSE114799,Samples (138),138
2,GSE38924,Samples (1),1
3,GSE91069,Samples (31),31
4,GSE6903,Samples (6),6
5,GSE125134,Samples (28),28
6,GSE121400,Samples (2),2
7,GSE66578,Samples (6),6
8,GSE13150,Samples (202),7
9,GSE123081,Samples (2),2


### Example GSE13150 - values did no match 

In [132]:
#GSE13150
#str.contains is getting another GSE where its substring is GSE13150. 

gsm_gse_geodb[gsm_gse_geodb['series_id'].str.contains('GSE13150', na=False)]



,gsm,series_id,gpl,GSE
139722,GSM156606,"GSE7045,GSE13150",GPL4661,GSE7045
139723,GSM156607,"GSE7045,GSE13150",GPL4661,GSE7045
139724,GSM156608,"GSE7045,GSE13150",GPL4661,GSE7045
139725,GSM156609,"GSE7045,GSE13150",GPL4661,GSE7045
139726,GSM156610,"GSE7045,GSE13150",GPL4661,GSE7045
139727,GSM156611,"GSE7045,GSE13150",GPL4661,GSE7045
139728,GSM156612,"GSE7045,GSE13150",GPL4661,GSE7045
139729,GSM156613,"GSE7045,GSE13150",GPL4661,GSE7045
139730,GSM156614,"GSE7045,GSE13150",GPL4661,GSE7045
139732,GSM156616,"GSE7045,GSE13150",GPL4661,GSE7045


In [222]:
#we have exactaly 202 samples. The problem is that str.contains get a substring - 12 more rows (GSE131503!)

for gse in df_GSE13150['series_id']:
    if 'GSE13150' in gse:
        print(gse)

GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE131

# Comparing values

In [224]:
#Transform Samples_NCBI string in values 

df_merged['samples_ncbi'] = df_merged['Samples_NCBI'].str.strip().str.split(' ').str.get(1).str.replace('(', ' ').str.replace(')', ' ').astype(int)

df_merged_final = df_merged.drop(columns=['Samples_NCBI']) #removing our string column


df_merged_final #ok

,GSE,samples_GEOdb,samples_ncbi
0,GSE76110,16,16
1,GSE114799,138,138
2,GSE38924,1,1
3,GSE91069,31,31
4,GSE6903,6,6
5,GSE125134,28,28
6,GSE121400,2,2
7,GSE66578,6,6
8,GSE13150,7,202
9,GSE123081,2,2


In [162]:
#Check the GSE that not matched with the NCBI because of the limitation of position. Just 9 GSE from 500

df_merged_final['values_checked'] = np.where(df_merged_final['samples_ncbi']!=df_merged_final['samples_GEOdb'], 
                                           'yes', 'no')


df_merged_final[df_merged_final['values_checked'].str.contains('yes')]

,GSE,samples_GEOdb,samples_ncbi,values_checked
8,GSE13150,7,202,yes
30,GSE89728,6,22,yes
164,GSE22997,9,13,yes
227,GSE35953,22,35,yes
339,GSE11318,243,406,yes
355,GSE112217,2,10,yes
363,GSE37954,10,18,yes
372,GSE10893,206,504,yes
380,GSE68387,486,2666,yes


In [217]:
#Creating a df specific to yes match - like print above
gse_9_yes = df_merged_final[df_merged_final['values_checked'].str.contains('yes')]


#put the 9 GSE above into a list
gse_9_list = gse_9_yes['GSE'].tolist()

gse_9_list


['GSE13150',
 'GSE89728',
 'GSE22997',
 'GSE35953',
 'GSE11318',
 'GSE112217',
 'GSE37954',
 'GSE10893',
 'GSE68387']

In [213]:
#example: get the GSE from gse_9_list present in all table

df = gsm_gse_geodb[gsm_gse_geodb['series_id'].isin(gse_9_list)].reset_index(drop=True)

unique_df = df.drop_duplicates('GSE')
list_unique = unique_df['GSE'].tolist()
list_unique

#'GSE11318' 'GSE10893' 'GSE22997' 'GSE35953' 'GSE37954'

['GSE11318', 'GSE10893', 'GSE22997', 'GSE35953', 'GSE37954']

In [181]:
#we have exactaly 202 samples. The problem is that str.contains get a substring - 12 more rows (GSE131503!)
#GSE13150 not unique

for gse in df_GSE13150['series_id']:
    if 'GSE13150' in gse:
        print(gse)
        
    

GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE13150
GSE7045,GSE131

In [223]:
#Another example - GSE89728
#22 samples when we looking for in series_id

gsm_gse_geodb[gsm_gse_geodb['series_id'].str.contains('GSE89728', na=False)]


,gsm,series_id,gpl,GSE
2397213,GSM2387316,"GSE89724,GSE89728",GPL11154,GSE89724
2397214,GSM2387317,"GSE89724,GSE89728",GPL11154,GSE89724
2397215,GSM2387318,"GSE89724,GSE89728",GPL11154,GSE89724
2397216,GSM2387319,"GSE89724,GSE89728",GPL11154,GSE89724
2397217,GSM2387320,"GSE89724,GSE89728",GPL11154,GSE89724
2397218,GSM2387321,"GSE89724,GSE89728",GPL11154,GSE89724
2397219,GSM2387328,"GSE89726,GSE89728",GPL16791,GSE89726
2397220,GSM2387329,"GSE89726,GSE89728",GPL16791,GSE89726
2397221,GSM2387330,"GSE89726,GSE89728",GPL16791,GSE89726
2397222,GSM2387331,"GSE89726,GSE89728",GPL16791,GSE89726


In [226]:
#GSE89728
#just 6 GSM, as in our merged table, because the problem of second term.
#But, the numbers are Ok with GEOMETADB

gsm_gse_geodb[gsm_gse_geodb['GSE'].str.contains('GSE89728', na=False)]


,gsm,series_id,gpl,GSE
2397225,GSM2404899,"GSE89728,GSE90495",GPL11154,GSE89728
2402088,GSM2642522,"GSE89728,GSE99346",GPL18573,GSE89728
2402089,GSM2642523,"GSE89728,GSE99346",GPL18573,GSE89728
2402090,GSM2642524,"GSE89728,GSE99346",GPL18573,GSE89728
2402091,GSM2642525,"GSE89728,GSE99346",GPL18573,GSE89728
2402092,GSM2642526,"GSE89728,GSE99346",GPL18573,GSE89728


In [225]:
#GSE89728

df_merged_final[df_merged_final['GSE'].str.contains('GSE89728', na=False)]

,GSE,samples_GEOdb,samples_ncbi
30,GSE89728,6,22


In [228]:
#GSE22997 - 13

gsm_gse_geodb[gsm_gse_geodb['series_id'].str.contains('GSE22997', na=False)]

,gsm,series_id,gpl,GSE
521013,GSM567521,"GSE22996,GSE22997",GPL6103,GSE22996
521014,GSM567522,"GSE22996,GSE22997",GPL6103,GSE22996
521015,GSM567523,"GSE22996,GSE22997",GPL6103,GSE22996
521016,GSM567524,"GSE22996,GSE22997",GPL6103,GSE22996
521020,GSM567532,GSE22997,GPL6103,GSE22997
521021,GSM567533,GSE22997,GPL6103,GSE22997
521022,GSM567534,GSE22997,GPL6103,GSE22997
521023,GSM567535,GSE22997,GPL6103,GSE22997
521024,GSM567536,GSE22997,GPL6103,GSE22997
521025,GSM567537,GSE22997,GPL6103,GSE22997


In [229]:
#GSE22997 - 9 samples
gsm_gse_geodb[gsm_gse_geodb['GSE'].str.contains('GSE22997', na=False)]


,gsm,series_id,gpl,GSE
521020,GSM567532,GSE22997,GPL6103,GSE22997
521021,GSM567533,GSE22997,GPL6103,GSE22997
521022,GSM567534,GSE22997,GPL6103,GSE22997
521023,GSM567535,GSE22997,GPL6103,GSE22997
521024,GSM567536,GSE22997,GPL6103,GSE22997
521025,GSM567537,GSE22997,GPL6103,GSE22997
521026,GSM567538,GSE22997,GPL6103,GSE22997
521027,GSM567539,GSE22997,GPL6103,GSE22997
521028,GSM567540,GSE22997,GPL6103,GSE22997


In [230]:
#GSE22997
df_merged_final[df_merged_final['GSE'].str.contains('GSE22997', na=False)]

,GSE,samples_GEOdb,samples_ncbi
164,GSE22997,9,13
